***GENERATED CODE FOR 1110gm PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import pandas as pd
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        hdfs = HDFileSystem(host=hdfs_server, port=hdfs_port)
        #hdfs = HDFileSystem(host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            dfPd = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CUST_ID', 'transformation_label': 'String Indexer'}], 'feature': 'CUST_ID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': 'C10001', 'max': 'C10521', 'missing': '0', 'distinct': '500'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CUST_ID'}, {'feature_label': 'CUST_ID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CUST_ID')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
import json
import optuna
import numpy as np
from sklearn import metrics
import pandas as pd
from sklearn.mixture import GaussianMixture


def gmm(df, TrainingPercent, k):
    rows = df.count()
    percentrow = TrainingPercent / 100 * rows
    percentrows = int(percentrow)
    df = df.limit(percentrows)
    pd_df = df.toPandas()

    def objective(trial):
        n_components = int(trial.suggest_loguniform("n_components", 2, 10))
        classifier_obj = GaussianMixture(n_components=n_components).fit(pd_df)
        labels = classifier_obj.predict(pd_df)
        score = metrics.silhouette_score(pd_df, labels, metric='euclidean')
        return score
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=10)
    best = study.best_params
    best_param = int(best["n_components"])
    model = GaussianMixture(n_components=k).fit(pd_df)
    labels = model.predict(pd_df)
    silhouette = metrics.silhouette_score(pd_df, labels, metric='euclidean')
    pd_df['prediction'] = pd.DataFrame(labels)
    labels_unique = np.unique(labels)
    totalClusters = len(labels_unique)
    totalClusters = model.summary.k
    display("totalClusters     : %s" % totalClusters)
    display("optimal number of clusters  : %s" % best_param)
    return pd_df


class Clustering:

    def run(spark_DF, spark, config):
        stageAttributes = json.loads(config)
        stageAttributes['model']
        trainingPercent = stageAttributes['TrainingPercent']
        clusters = stageAttributes['totalClusters']

        gmmmeans_model = gmm(
            df=spark_DF, TrainingPercent=trainingPercent, k=clusters)


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run 1110gmHooks.ipynb
try:
	#sourcePreExecutionHook()

	ccgeneralsample = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/testdata/1697026176276_CC_GENERAL_Sample.csv', 'filename': 'CC_GENERAL_Sample.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'FilePath': '/ManualPipelines/Clustering_Manual/CC_GENERAL_Sample.csv', 'viewFileName': 'CC_GENERAL_Sample.csv', 'is_header': 'Use Header Line', 'baseType': 'hdfs', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/', 'results_url': 'http://ml.numtra.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run 1110gmHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(ccgeneralsample,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "CUST_ID", "transformation_label": "String Indexer"}], "feature": "CUST_ID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "C10001", "max": "C10521", "missing": "0", "distinct": "500"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CUST_ID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "BALANCE", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "2468.28", "stddev": "2831.33", "min": "0.0", "max": "19043.13856", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "BALANCE"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "BALANCE_FREQUENCY", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.9", "stddev": "0.22", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "BALANCE_FREQUENCY"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PURCHASES", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1443.12", "stddev": "2609.87", "min": "0.0", "max": "32539.78", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PURCHASES"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ONEOFF_PURCHASES", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "845.84", "stddev": "1884.43", "min": "0.0", "max": "26547.43", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "ONEOFF_PURCHASES"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "INSTALLMENTS_PURCHASES", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "597.28", "stddev": "1221.03", "min": "0.0", "max": "12560.85", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "INSTALLMENTS_PURCHASES"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CASH_ADVANCE", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1245.28", "stddev": "2692.78", "min": "0.0", "max": "27296.48576", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "CASH_ADVANCE"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PURCHASES_FREQUENCY", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.51", "stddev": "0.4", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PURCHASES_FREQUENCY"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ONEOFF_PURCHASES_FREQUENCY", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.26", "stddev": "0.33", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "ONEOFF_PURCHASES_FREQUENC..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PURCHASES_INSTALLMENTS_FREQUENCY", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.37", "stddev": "0.4", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PURCHASES_INSTALLMENTS_FR..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CASH_ADVANCE_FREQUENCY", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.13", "stddev": "0.2", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "CASH_ADVANCE_FREQUENCY"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CASH_ADVANCE_TRX", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "3.12", "stddev": "5.7", "min": "0", "max": "40", "missing": "0"}, "updatedLabel": "CASH_ADVANCE_TRX"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PURCHASES_TRX", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "18.64", "stddev": "30.15", "min": "0", "max": "229", "missing": "0"}, "updatedLabel": "PURCHASES_TRX"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "CREDIT_LIMIT", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "6329.2", "stddev": "4419.29", "min": "500", "max": "23000", "missing": "0"}, "updatedLabel": "CREDIT_LIMIT"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PAYMENTS", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "2417.54", "stddev": "3698.6", "min": "0.0", "max": "32048.6122", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PAYMENTS"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "MINIMUM_PAYMENTS", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1167.75", "stddev": "2374.83", "min": "0.0", "max": "27631.45072", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "MINIMUM_PAYMENTS"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PRC_FULL_PAYMENT", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.12", "stddev": "0.27", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PRC_FULL_PAYMENT"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "TENURE", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "11.76", "stddev": "0.94", "min": "6", "max": "12", "missing": "0"}, "updatedLabel": "TENURE"}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run 1110gmHooks.ipynb
try:
	#mlPreExecutionHook()

	gaussionmixture = Clustering.run(autofe,spark,json.dumps( {"autoClustering": 0, "originalfile": "/FileStore/platform/sampleData/65269dfb0f0167dffb21aa30/part-00000-e482d342-83ce-46e6-a2c0-c2055e7da96e-c000.csv", "model": "GaussionMixture", "TrainingPercent": 100, "totalClusters": 3}))

	#mlPostExecutionHook(gaussionmixture)

except Exception as ex: 
	logging.error(ex)
#spark.stop()
